In [1]:
%cd ../src

/media/alex/Data/Backup_root/Backup_docs/git/reddit-sarc/src


In [2]:
from rnn_util import *
from util import *

In [3]:
with open(FULL_COMMENTS, 'r') as f:
    comments = json.load(f)
print(len(comments))
print(type(comments))

12704751
<class 'dict'>


In [4]:
from itertools import islice
from random import shuffle
from copy import copy

comment_sets = []
with open(FULL_TRAIN_BALANCED, 'r') as f:
    reader = csv.reader(f, delimiter='|')
    for row in reader:
        #ancestors_idx = row[0].split(' ')
        responses_idx = row[1].split(' ')
        comment_sets.append(responses_idx)
        #labels = np.array(row[2].split(' '), dtype=np.int32)

#keys = [k for k in comments.keys()]
#shuffle(keys)

shuffle(comment_sets)
num_sets = len(comment_sets)

train_idx = int(num_sets*.9)
val_idx = int(num_sets*.95)
train_sets, val_sets, holdout_sets = comment_sets[:train_idx], comment_sets[train_idx : val_idx], comment_sets[val_idx :]

seen_users = set()
for s in train_sets:
    for k in s:
        v = comments[k]
        seen_users.add(v['author'])
    
print('Num users seen in train set: ', len(seen_users))
    
num_unseen_val = 0
num_val = 0
for s in val_sets:
    for k in s:
        v = comments[k]
        if not v['author'] in seen_users: num_unseen_val += 1
        num_val += 1

print('Num unseen val {} out of {}; proportion is {}'.format(num_unseen_val, num_val, num_unseen_val / num_val))

num_unseen_holdout = 0
num_holdout = 0
for s in holdout_sets:
    for k in s:
        v = comments[k]
        if not v['author'] in seen_users: num_unseen_holdout += 1
        num_holdout += 1
    
print('Num unseen holdout {} out of {}; proportion is {}'.format(num_unseen_holdout, num_holdout, num_unseen_holdout / num_holdout))

num_unseen_test = 0
num_test = 0
with open(POL_TEST_UNBALANCED, 'r') as f:
    reader = csv.reader(f, delimiter='|')
    for row in reader:
        responses_idx = row[1].split(' ')
        for k in responses_idx:
            v = comments[k]
            if not v['author'] in seen_users: num_unseen_test += 1
            num_test += 1
print('Num unseen test {} out of {}; proportion is {}'.format(num_unseen_test, num_test, num_unseen_test / num_test))



Num users seen in train set:  168304
Num unseen val 7535 out of 12854; proportion is 0.5861988486074374
Num unseen holdout 7637 out of 12856; proportion is 0.5940416925948974
Num unseen test 43544 out of 78425; proportion is 0.5552311125278929


In [7]:
from test_configs import *

print("Differences in politics balanced configs:")
for k in B2.keys():
    if not (B2[k] == B3[k] == B4[k]):
        print(k)
        

print("\n\nDifferences in full balanced configs:")
for k in C2.keys():
    if not (C2[k] == C3[k] == C4[k]):
        print(k)

Differences in politics balanced configs:
hidden_dim
dropout
l2_lambda
second_linear_layer


Differences in full balanced configs:
hidden_dim
dropout
l2_lambda
second_linear_layer


In [3]:
%cd ../src
from rnn_util import *
from test_configs import *

/media/alex/Data/Backup_root/Backup_docs/git/reddit-sarc/src


In [4]:
fasttext_lookup, fasttext_word_to_idx = load_embeddings_by_index(FASTTEXT_FILE)

In [7]:
dataset = build_and_split_dataset(word_to_idx=fasttext_word_to_idx, **B2)

In [9]:
first_holdout = list(dataset['raw_holdouts'].keys())[0]
print(first_holdout)

0.05 holdout


In [15]:
dataset['raw_holdouts'][first_holdout][0]

{'ancestors': ['Here\'s a lovely story: A pair of rich assholes move into one of the most popular ballooning spots in the country, build a gigantic "Moorish fortress castle", &amp; then start suing the balloonists out of existence because they don\'t like people flying over their exclusive getaway property.'],
 'responses': ['See, the rich ARE job creators.',
  "By that logic couldn't I sue the US Navy for all the planes that fly over my house on the way to and from the Naval air station near by?"],
 'labels': array([1, 0], dtype=int32),
 'ancestor_authors': ['anutensil'],
 'response_authors': ['fingers', 'njm1314'],
 'ancestor_subreddits': ['politics'],
 'response_subreddits': ['politics', 'politics']}

In [28]:
first_example = dataset['holdout_datas'][first_holdout]['X'][0]
second_example = dataset['holdout_datas'][first_holdout]['X'][1]

In [31]:
fasttext_idx_to_word = {idx:word for word,idx in fasttext_word_to_idx.items()}
fasttext_idx_to_word[0] = 'UNK'


In [50]:
def decode(ex):
    ex = [int(x) for x in ex]
    num_trailing_zeroes = 0
    for i in range(1, len(ex)):
        if ex[-i] == 0: num_trailing_zeroes = i
        else: break
    ex = ex[:len(ex) - num_trailing_zeroes]
    return ' '.join([fasttext_idx_to_word[x] for x in ex])

In [51]:
decode(second_example)

"By that logic couldn 't I sue the US Navy for all the planes that fly over my house on the way to and from the Naval air station near by ?"